1. 数据预处理
  - 去掉不重要的对模型结果影响较小的特征
  - 去掉一些明显没用的特征
  - 确定当前贷款状态(label值)
  - 去掉特征中只有一种属性的列
  - 处理缺失值
  - 数据类型的转换
    > 由于sk-learn 库不接受字符型的数据，所有还需将上面特征中字符型的数据进行处理

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
# 加载数据集 跳过第一行
loans_2007 = pd.read_csv('data/LoanStats3a.csv', skiprows=1)
len(loans_2007)

39535

In [2]:
half_count = len(loans_2007) / 2
print(loans_2007.shape)

(39535, 111)


In [3]:
# 删除
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)
print(loans_2007.shape)

(39535, 54)


In [4]:
loans_2007 = loans_2007.drop(["desc", "url"], axis=1)
print(loans_2007.shape)
# 保存数据
loans_2007.to_csv('loans_2007.csv', index=False)

(39535, 52)


In [5]:
# 重新加载数据
import pandas as pd
loans_2007 = pd.read_csv("loans_2007.csv")
print(loans_2007.iloc[0])
print("-"*30)
print(loans_2007.shape[1])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                            Dec-11
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

- id:是用户唯一ID，是谁其实不重要
- member_id：填写表的ID号
- loan_amnt：期望贷款的值 ，比如说期望贷款5000
- funded_amnt：期望贷款了多少钱
- funded_amnt_inv：实际上发放了多少钱，是模型建立后的预测值吧，对于咱们马上建立的模型是没有用的，把当时填的信息为准，而不是模型预测后的数据为准
- term：还款日期，分36个月 ，这个是可以用的
- int_rate：利息，10.65% ，利息越高，还款越困难，反之，越容易，这个可以用
- grade ：B ， 打分 ，其他信贷机构打的分
- sub_grade ：B2 ，打分  ，其实打分都是说的是一个事，去掉一个就行了，比如有两列都是给客户打分
- emp_title：客户所在公司名称，越有名气越好，贷款越容易，可以把公司分成A B C D的档次，耗时和成本太高，把成千上万家公司分成，难度很大，除非已经分好了,所以这个特征就不要了
- home_ownership(房屋的使用权) ：RENT ，房屋是租的、还是你自己的的还是贷款买的，这个可以当做特征
- annual_inc:客户所在的公司年度增长额度
- verification_status：身份证是否被核实，Verified核实
- issue_d：逾期时间 ，Dec-11 12月11日

In [6]:

#删除：客户ID和被模型预测后的值
#客户唯一的ID、编号ID、loan_amnt：期望贷款的值、funded_amnt_inv：实际上发放了多少钱、grade和sub_grade给客户打分
#emp_title：客户所在公司名称
#issue_d：逾期时间
loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)

In [7]:
"""
#删除：被模型预测后的值
#zip_code：邮编 美国常用
#out_prncp和out_prncp_inv都是一样的：总资金中剩余的未偿还本金
#out_prncp_inv：实际未偿还的本金
#total_rec_prncp：迄今收到的本金
"""
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)


In [8]:

"""
#删除：被模型预测后的值
#total_rec_int：迄今收到的利息

#recoveries：是否收回本金
#collection_recovery_fee：收集回收费用
#last_pymnt_d：最近一次收到还款的时间
#last_pymnt_amnt：全部的还款的时间
"""

#保留候选特征
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans_2007.iloc[0])#第一行数据
print("-----------------------")
print(loans_2007.shape[1]) #原始是52列，现在是32列的候选特征，还是有点“冗余”，但是还少了一个东西呀，是啥样.....是不是“标签label”啊

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                   Jan-85
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                         83.70%
total_acc                               9
initial_list_status                     f
last_credit_pull_d                

In [9]:
loans_2007['loan_status'].value_counts()

Fully Paid            33693
Charged Off            5612
Current                 201
Late (31-120 days)       10
In Grace Period           9
Late (16-30 days)         5
Default                   1
Name: loan_status, dtype: int64

In [10]:
# 寻找标签值， 用 0. 1来表示
loans_2007 = loans_2007[
    (loans_2007['loan_status'] == "Fully Paid") |
    (loans_2007['loan_status'] == "Charged Off")
]
status_replace = {
    "loan_status": {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}
# replace:执行的是查找并替换的操作
loans_2007 = loans_2007.replace(status_replace)

In [11]:
# 去掉特征中只有一种属性或者特征的列，对于分类模型的预测是没有用的
orig_columns = loans_2007.columns # 展现所有的列
drop_columns = [] # 初始化空值
for col in orig_columns:
    # dropna() 先删除空值，在去重算唯一的属性
    col_series = loans_2007[col].dropna().unique()
    # 如果该特征只有一个属性，就给过滤掉该特征
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
display(loans_2007.shape)
loans_2007.to_csv("filtered_loans_2007.csv", index=False)

(39305, 24)

In [12]:
# 处理缺失值
import pandas as pd
loans = pd.read_csv('filtered_loans_2007.csv')
null_counts = loans.isnull().sum()
print(null_counts)

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1073
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
pymnt_plan                 0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         1
pub_rec_bankruptcies     449
dtype: int64


In [13]:
loans['emp_length'].value_counts()

10+ years    8796
< 1 year     4505
2 years      4344
3 years      4056
4 years      3406
5 years      3269
1 year       3189
6 years      2205
7 years      1751
8 years      1465
9 years      1246
Name: emp_length, dtype: int64

In [14]:
# pub_rec_bankruptcies：不良记录数
loans = loans.drop("pub_rec_bankruptcies", axis=1)
loans = loans.dropna(axis=0)
loans.dtypes.value_counts()

object     12
float64    10
int64       1
dtype: int64

In [15]:
# 数据类型的转换
object_columns_df = loans.select_dtypes(include=["object"])
object_columns_df.iloc[0]

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
pymnt_plan                       n
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line            Jan-85
revol_util                  83.70%
last_credit_pull_d          Nov-16
Name: 0, dtype: object

- term: 分期多少个月啊
- int_rate: 利息， 10.65%，后面还要吧%去掉。
- emp_length: 10年的映射成10， 9年的映射成9
- home_ownership: 房屋所有权，是租的， 自己的，还是抵押出去了，就用 0 1 2 来代替

In [16]:
#查看指定标签的属性，并记数
#home_ownership：房屋所有权
#verification_status：身份保持证明
#emp_length：客户公司名称
#term：贷款分期的时间
#addr_state：地址邮编
cols = [
    'home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state'
]
for c in cols:
    print(loans[c].value_counts())

RENT        18237
MORTGAGE    17035
OWN          2805
OTHER          96
NONE            1
Name: home_ownership, dtype: int64
Not Verified       16182
Verified           12251
Source Verified     9741
Name: verification_status, dtype: int64
10+ years    8794
< 1 year     4492
2 years      4339
3 years      4052
4 years      3397
5 years      3262
1 year       3182
6 years      2201
7 years      1747
8 years      1463
9 years      1245
Name: emp_length, dtype: int64
 36 months    27980
 60 months    10194
Name: term, dtype: int64
CA    6876
NY    3644
FL    2739
TX    2657
NJ    1799
IL    1478
PA    1470
VA    1355
GA    1342
MA    1278
OH    1176
MD    1019
AZ     824
WA     788
CO     758
NC     739
CT     725
MI     688
MO     654
MN     589
NV     477
SC     457
OR     431
WI     429
AL     424
LA     422
KY     320
OK     292
KS     257
UT     248
AR     233
DC     211
RI     196
NM     180
HI     168
WV     167
NH     160
DE     109
MT      78
WY      78
AK      77
SD      61
VT  

In [17]:
# purpoes 和 title 表达的意思相近，且从输出结果可以看出来 title 所含的属性较多， 可以将其舍弃掉
print(loans["purpose"].value_counts()) # 贷款目的
print("-"*30)
print(loans["title"].value_counts()) # title: 跟purpose 任选其一

debt_consolidation    18057
credit_card            4927
other                  3761
home_improvement       2846
major_purchase         2103
small_business         1745
car                    1489
wedding                 924
medical                 665
moving                  551
house                   364
vacation                347
educational             300
renewable_energy         95
Name: purpose, dtype: int64
------------------------------
Debt Consolidation                                 2122
Debt Consolidation Loan                            1670
Personal Loan                                       625
Consolidation                                       502
debt consolidation                                  483
                                                   ... 
Loan2011                                              1
Hidden Creek                                          1
business expansion                                    1
home improvement project excellent credit!!!!

In [18]:
# labelencoder
# jemp_length 做成字典，emp_length 当做key, value里还是字典， "10+ years": 10 ...
# 再在后面调用replace函数，刚才利息这列特征，有% 可以用 astype() 处理一下
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
# 删除：last_credit_pull_d：LC撤回最近的月份   
#earliest_cr_line：第一次借贷时间
#addr_state：家庭邮编
#title：URL的标题
loans = loans.drop(
    ["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
#rstrip()：删除 string 字符串末尾的指定字符
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
#revol_util：透支额度占信用比例
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

- 剩余的其他字符型特征，此处选择使用 pandas 的 get_dummies() 函数，直接映射为数值型.

In [19]:
#查看指定标签的属性，并记数
#home_ownership：房屋所有权
#verification_status：身份保持证明
#emp_length：客户公司名称
#purpose：贷款的意图
#term：贷款分期的时间

cat_columns = ["home_ownership", "verification_status", "emp_length", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)
loans = loans.drop("pymnt_plan", axis=1)

In [20]:
loans.to_csv('cleaned_loans2007.csv', index=False)

###  什么时候用 oneHotEncoder独热编码和LabelEncoder标签编码？
- 特征的属性小于等于3，就用OneHotEncoder, 比如说: 天气， 性别， 属于无序特征
- 特征的属性大于3， 就用LabelEncoder, 比如说: 星期，属于有序型

In [36]:
import pandas as pd
loans = pd.read_csv("cleaned_loans2007.csv")
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38174 entries, 0 to 38173
Data columns (total 48 columns):
loan_amnt                              38174 non-null float64
int_rate                               38174 non-null float64
installment                            38174 non-null float64
annual_inc                             38174 non-null float64
loan_status                            38174 non-null int64
dti                                    38174 non-null float64
delinq_2yrs                            38174 non-null float64
inq_last_6mths                         38174 non-null float64
open_acc                               38174 non-null float64
pub_rec                                38174 non-null float64
revol_bal                              38174 non-null float64
revol_util                             38174 non-null float64
total_acc                              38174 non-null float64
home_ownership_MORTGAGE                38174 non-null int64
home_ownership_NONE        

In [77]:
import matplotlib.pyplot as plt
loans_corr_df = loans.corr()[['loan_status']]
loans_corr_df["abs_loan_status"] = abs(loans_corr_df['loan_status'])
loans_corr_df = loans_corr_df.sort_values('loan_status', ascending=False).drop(columns=['abs_loan_status'])
# plt.rcParams["font.family"] = "AR PL UKai CN"
# plt.rcParams["axes.unicode_minus"] = False
# plt.rcParams["font.size"] = 12
# plt.figure(figsize=(20, 8), dpi=80)
# indexs = loans_corr_df.index.to_list()[0:10]
# plt.bar(range(len(indexs)), loans_corr_df['loan_status'].to_list()[0:10])
# plt.ylim(-1, 1)
# plt.xticks(range(len(indexs)), indexs)
loans_corr_df[0:10]

,loan_status
loan_status,1.000000
term_ 36 months,0.156455
verification_status_Not Verified,0.042437
purpose_credit_card,0.040791
annual_inc,0.039482
purpose_major_purchase,0.028595
home_ownership_MORTGAGE,0.023519
purpose_home_improvement,0.021515
purpose_car,0.021515
total_acc,0.020633


In [80]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.linear_model import LogisticRegression
cols = loans.columns
train_cols = cols.drop("loan_status")
X = loans[train_cols]
y = loans['loan_status']
# 由大到小，进行降序
lr = LogisticRegression()
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=1)
train_model = lr.fit(train_X, train_y)
perm = PermutationImportance(train_model, random_state=1).fit(test_X, test_y)
eli5.show_weights(perm, feature_names=test_X.columns.tolist())

Weight,Feature
0.0559 ± 0.0034,loan_amnt
0.0193 ± 0.0023,installment
0.0025 ± 0.0010,annual_inc
0.0024 ± 0.0007,revol_bal
0.0004 ± 0.0002,dti
0.0003 ± 0.0008,total_acc
0.0002 ± 0.0002,open_acc
0.0001 ± 0.0001,int_rate
0.0000 ± 0.0002,revol_util
0 ± 0.0000,inq_last_6mths


In [23]:
# 自己尝试的.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
cols = loans.columns
train_cols = cols.drop("loan_status")
X = loans[train_cols]
y = loans['loan_status']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)
lr = LogisticRegression()
# lr.fit(X_train, y_train)
# y_hat = lr.predict(X_test)
lr.fit(X, y)
y_hat = lr.predict(X)
print("权重:", lr.coef_)
print("概率模型:", lr.predict_proba(X))
print("偏置:", lr.intercept_)
print("结果:", y_hat[:10]) # 0 表示没有偿还，1 表示偿还

权重: [[-9.73042111e-05  4.49402864e-03  3.05632610e-03  1.41968934e-05
   2.29139201e-02 -4.14712426e-05 -1.56999319e-04  1.33444663e-02
  -1.83992983e-04 -1.32734476e-05  3.77377500e-04  2.72017698e-02
   4.86226196e-04  5.61207136e-07 -3.36077351e-06  2.74002806e-04
   1.28197303e-03  1.67046402e-03  5.39558443e-04 -1.70619994e-04
   4.02846718e-04  2.44761081e-04  4.01756434e-04  2.75976775e-04
   2.17043822e-04  1.83565231e-04  1.26523331e-04  5.79396130e-05
   5.79951330e-05  7.14909399e-05 -4.96611229e-07  2.36238521e-04
   4.85079881e-04  6.60879763e-04  2.31472058e-05  1.75698355e-04
   2.98937731e-06  3.24733716e-04  3.22055752e-05  4.27192398e-05
   1.93875024e-04 -3.38131028e-06 -2.76195797e-04  2.64032733e-05
   1.15009643e-04  2.70930530e-03 -6.69902836e-04]]
概率模型: [[0.23597278 0.76402722]
 [0.35661543 0.64338457]
 [0.31573614 0.68426386]
 ...
 [0.3074582  0.6925418 ]
 [0.1029464  0.8970536 ]
 [0.09553652 0.90446348]]
偏置: [0.0020394]
结果: [1 1 1 1 1 1 1 1 1 1]


In [24]:
lr.predict_proba(X)

array([[0.23597278, 0.76402722],
       [0.35661543, 0.64338457],
       [0.31573614, 0.68426386],
       ...,
       [0.3074582 , 0.6925418 ],
       [0.1029464 , 0.8970536 ],
       [0.09553652, 0.90446348]])

In [25]:
# 建立混淆矩阵
import pandas as pd
# 接下来如何计算4个指标 fp tp fn tn

# 假正类(False Positive, FP): 将负类预测为正类
fp_filter = (y_hat == 1) & (loans['loan_status'] == 0)
fp = len(y_hat[fp_filter])
print(fp)
print("-"*30)

# 真正类(True Positive, TP): 将正类预测为正类
tp_filter = (y_hat == 1) & (loans['loan_status'] == 1)
tp = len(y_hat[tp_filter])
print(tp)
print("-"*30)

# 假负类(False Negative, FN): 将正类预测为负类
fn_filter = (y_hat == 0) & (loans['loan_status'] == 1)
fn = len(y_hat[fn_filter])
print(fn)
print("-"*30)

# 真负类(True Negative, TN): 将负类预测为负类
tn_filter = (y_hat == 0) & (loans["loan_status"] == 0)
tn = len(y_hat[tn_filter])
print(tn)

5355
------------------------------
32786
------------------------------
23
------------------------------
10


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression()
predictions = cross_val_predict(lr, X, y, cv=10)
predictions = pd.Series(predictions)
print(predictions[:1000])

0      1
1      1
2      1
3      1
4      1
      ..
995    1
996    1
997    1
998    1
999    1
Length: 1000, dtype: int64


In [27]:
# 假正类(False Positive, FP): 将负类预测为正类
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# 真正类(True Positive, TP): 将正类预测为正类
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# 假负类(False Negative, FN): 将负类预测为正类
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# 真负类(True Negative, TN): 将负类预测为负类
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

$$
    TRP = \frac{truepositives}{falsepositives + truepositives}
$$

$$
    FPR = \frac{falsepositives}{falsepositives + truepositives}
$$

In [28]:
# Rates: 就可以用刚才的指标进行衡量了啊
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))
print(tpr) # 真正率
print(fpr) # 假正率

0.9990246578682679
0.9985088536812675


**TPR 和 FPR的值都很高，说明一个人基本会判断为可以借钱，那显然模型就没有分类意义**

In [81]:
# 考虑权重后使用逻辑回归训练
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
"""
class_weight: 可以调整正反样本的权重
balanced: 希望正负样本平衡一些
"""
lr = LogisticRegression(class_weight="balanced")
predictions = cross_val_predict(lr, X, y, cv=10)
predictions = pd.Series(predictions)

# 假正类(False Positive, FP): 将负类预测为正类
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# 真正类(True Positive, TP): 将正类预测为正类
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# 假负类(False Negative, FN): 将负类预测为正类
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# 真负类(True Negative, TN): 将负类预测为负类
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))
print(f"真正率:{tpr}") # 真正率
print(f"假正率:{fpr}") # 假正率 

真正率:0.5325368039257521
假正率:0.33998136067101586


In [30]:
# 自定义权重后使用逻辑回归训练
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
"""
权重项可以自己定义
0代表5倍的
1代表10倍的
"""
penalty = {
    0:5,
    1:1
}
lr = LogisticRegression(class_weight=penalty)
kf = 10
predictions = cross_val_predict(lr, X, y, cv=kf)
predictions = pd.Series(predictions)

# 假正类(False Positive, FP): 将负类预测为正类
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# 真正类(True Positive, TP): 将正类预测为正类
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# 假负类(False Negative, FN): 将负类预测为正类
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# 真负类(True Negative, TN): 将负类预测为负类
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))

print(tpr)
print()
print(fpr)

0.6939254472858057

0.5090400745573159
